In [3]:
# %% [markdown]
# # BINN – Definición de datos (ENV + TAG) y One Big Table
# Este notebook define variables, esquemas (dtypes), construye ejemplos y genera la OBT final.

# %% [markdown]
# ## 0) Imports y utilidades

# %%
import pandas as pd
import numpy as np
from pathlib import Path

# reproducibilidad
rng = np.random.default_rng(123)

def enforce_dtypes(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    """Aplica dtypes declarados (schema) a un DataFrame."""
    for col, dt in schema.items():
        if col not in df.columns:
            continue
        if str(dt).startswith("category"):
            df[col] = df[col].astype("category")
        elif str(dt).startswith("datetime64"):
            df[col] = pd.to_datetime(df[col])
        else:
            df[col] = df[col].astype(dt)
    return df

def make_time_bins(n, start="2017-11-01", freq="7D"):
    start_dt = pd.to_datetime(start)
    return pd.date_range(start_dt, periods=n, freq=freq)

def rand_coords(n, lat_range=(-60, 60), lon_range=(-180, 180)):
    lat = rng.uniform(*lat_range, size=n).round(4)
    lon = rng.uniform(*lon_range, size=n).round(4)
    return lat.astype("float64"), lon.astype("float64")

def zscore(series: pd.Series) -> pd.Series:
    mu = series.mean()
    sd = series.std(ddof=0) + 1e-6
    return ((series - mu) / sd).astype("float32")


# %% [markdown]
# ## 1) Variables AMBIENTALES (ENV)
# **Lista de variables:**
# - `CHL_raw`  : Clorofila-a
# - `SST_raw`  : Temperatura superficial del mar
# - `dSST_raw` : Magnitud del gradiente |∇SST|
# - `EKE_raw`  : Eddy Kinetic Energy
# - `Depth_raw`: Batimetría (profundidad)
# - `Light_raw`: Luz/Daylength/PAR (normalizado)
# - `Effort`   : Esfuerzo de muestreo/detectabilidad
# - `S_maxent` : Prior / idoneidad de MAXENT en (x,t)
#
# **Esquema esperado por dataset** (uno por variable):
# `{"lat":"float32","lon":"float32","time_bin":"datetime64[ns]","<VAR>":"float32"}`

# %%
# Definición de esquemas ENV (uno por variable)
schema_env_generic = {"lat":"float32","lon":"float32","time_bin":"datetime64[ns]"}
schema_env_chl   = {**schema_env_generic, "CHL_raw":"float32"}
schema_env_sst   = {**schema_env_generic, "SST_raw":"float32"}
schema_env_dsst  = {**schema_env_generic, "dSST_raw":"float32"}
schema_env_eke   = {**schema_env_generic, "EKE_raw":"float32"}
schema_env_depth = {**schema_env_generic, "Depth_raw":"float32"}
schema_env_light = {**schema_env_generic, "Light_raw":"float32"}
schema_effort    = {**schema_env_generic, "Effort":"float32"}
schema_maxent    = {**schema_env_generic, "S_maxent":"float32"}

def make_env_df(var_name: str, n=10, start="2017-11-01", low=0.0, high=1.0) -> pd.DataFrame:
    lat, lon = rand_coords(n)
    t = make_time_bins(n, start=start, freq="7D")
    val = rng.uniform(low, high, size=n).astype("float64")
    df = pd.DataFrame({"lat": lat, "lon": lon, "time_bin": t, var_name: val})
    # aplica dtypes
    schema = {
        "CHL_raw":   schema_env_chl,
        "SST_raw":   schema_env_sst,
        "dSST_raw":  schema_env_dsst,
        "EKE_raw":   schema_env_eke,
        "Depth_raw": schema_env_depth,
        "Light_raw": schema_env_light,
        "Effort":    schema_effort,
        "S_maxent":  schema_maxent,
    }[var_name]
    return enforce_dtypes(df, schema)

# Datasets ENV de ejemplo (10 filas)
env_chl   = make_env_df("CHL_raw",   n=10, low=0.03, high=5.0)
env_sst   = make_env_df("SST_raw",   n=10, low=16.0, high=30.0)
env_dsst  = make_env_df("dSST_raw",  n=10, low=0.0,  high=2.0)
env_eke   = make_env_df("EKE_raw",   n=10, low=0.0,  high=0.6)
env_depth = make_env_df("Depth_raw", n=10, low=5,    high=5000)
env_light = make_env_df("Light_raw", n=10, low=0.0,  high=1.0)
effort_map= make_env_df("Effort",    n=10, low=0.01, high=1.0)
maxent_s  = make_env_df("S_maxent",  n=10, low=0.1,  high=0.9)

print("ENV – CHL_raw (10 filas):\n", env_chl.head(), "\n")
print("ENV – SST_raw (10 filas):\n", env_sst.head(), "\n")


# %% [markdown]
# ## 2) Datos del TAG
# Todas las **variables del TAG** van en **una sola tabla**.
#
# **Variables crudas**:
# - tiempo/geo: `date`, `latitude`, `longitude`, (opcional `time_h`)
# - sensores: `pressure_dbar`, `depth_m`, `temperature_C`
# - acelerómetro: `accel_x`, `accel_y`, `accel_z`
# - velocidad: `vel_x`, `vel_y`, `vel_z`
# - química: `pH`
# - magnetómetro: `mag_x`, `mag_y`, `mag_z`
# - estado del tag: `battery_soc_%`, `capacitive`
#
# **Derivadas (por muestra)**:
# - `odba` (|ax|+|ay|+|az|), `speed_ms` (norma de vel), `heading_deg` (atan2(mag_y, mag_x))
#
# **Esquema esperado (TAG – una tabla):**
# ```
# {
#  "time_h":"float32", "date":"datetime64[ns]",
#  "latitude":"float32","longitude":"float32",
#  "pressure_dbar":"float32","depth_m":"float32","temperature_C":"float32",
#  "accel_x":"float32","accel_y":"float32","accel_z":"float32",
#  "vel_x":"float32","vel_y":"float32","vel_z":"float32",
#  "pH":"float32",
#  "mag_x":"float32","mag_y":"float32","mag_z":"float32",
#  "battery_soc_%":"float32","capacitive":"float32",
#  "odba":"float32","speed_ms":"float32","heading_deg":"float32"
# }
# ```

# %%
# Esquema TAG
schema_tag = {
    "time_h":"float32",
    "date":"datetime64[ns]",
    "latitude":"float32",
    "longitude":"float32",
    "pressure_dbar":"float32",
    "depth_m":"float32",
    "temperature_C":"float32",
    "accel_x":"float32","accel_y":"float32","accel_z":"float32",
    "vel_x":"float32","vel_y":"float32","vel_z":"float32",
    "pH":"float32",
    "mag_x":"float32","mag_y":"float32","mag_z":"float32",
    "battery_soc_%":"float32",
    "capacitive":"float32",
    "odba":"float32","speed_ms":"float32","heading_deg":"float32"
}

# Datos TAG de ejemplo (10 filas)
n_tag = 10
time_h = np.arange(n_tag, dtype=float)
dates = pd.date_range("2017-11-02", periods=n_tag, freq="12H")
lat_tag, lon_tag = rand_coords(n_tag)

tag = pd.DataFrame({
    "time_h": time_h,
    "date": dates,
    "latitude": lat_tag,
    "longitude": lon_tag,
    "pressure_dbar": rng.uniform(0, 250, size=n_tag),
    "depth_m": rng.uniform(0, 250, size=n_tag),
    "temperature_C": rng.uniform(10, 28, size=n_tag),
    "accel_x": rng.normal(0, 1, size=n_tag),
    "accel_y": rng.normal(0, 1, size=n_tag),
    "accel_z": rng.normal(0, 1, size=n_tag),
    "vel_x": rng.normal(0, 0.5, size=n_tag),
    "vel_y": rng.normal(0, 0.5, size=n_tag),
    "vel_z": rng.normal(0, 0.5, size=n_tag),
    "pH": rng.uniform(7.6, 8.2, size=n_tag),
    "mag_x": rng.normal(0, 50, size=n_tag),
    "mag_y": rng.normal(0, 50, size=n_tag),
    "mag_z": rng.normal(0, 50, size=n_tag),
    "battery_soc_%": rng.uniform(30, 100, size=n_tag),
    "capacitive": rng.uniform(0, 1, size=n_tag)
})

# Derivadas
tag["odba"] = (tag[["accel_x","accel_y","accel_z"]].abs().sum(axis=1)).astype("float64")
tag["speed_ms"] = (np.sqrt((tag[["vel_x","vel_y","vel_z"]]**2).sum(axis=1))).astype("float64")
tag["heading_deg"] = (np.degrees(np.arctan2(tag["mag_y"], tag["mag_x"]))).astype("float64")

# Aplica dtypes
tag = enforce_dtypes(tag, schema_tag)

print("TAG – tabla única (10 filas):\n", tag.head(), "\n")


# %% [markdown]
# ## 3) Agregación TAG → (lat, lon, time_bin) y unión con ENV
# - Definimos `time_bin` semanal (`W-MON`).
# - Agregamos TAG por (lat, lon, time_bin).
# - Fusionamos todas las variables ENV en esa misma clave.
# - Calculamos z-scores de las variables ENV (para el modelo).
#
# **Esquema esperado del TAG agregado (por bin):**
# ```
# {
#  "lat":"float32","lon":"float32","time_bin":"datetime64[ns]",
#  "pressure_dbar":"float32","depth_m":"float32","temperature_C":"float32",
#  "odba":"float32","speed_ms":"float32","heading_deg":"float32",
#  "pH":"float32","battery_soc_%":"float32","capacitive":"float32"
# }
# ```

# %%
# Armar (lat, lon, time_bin)
tag["lat"] = tag["latitude"].round(4).astype("float32")
tag["lon"] = tag["longitude"].round(4).astype("float32")
tag["time_bin"] = pd.to_datetime(tag["date"]).dt.to_period("W-MON").apply(lambda p: p.start_time)

# Agregación TAG por bin
tag_bin = (tag.groupby(["lat","lon","time_bin"], as_index=False)
           .agg({
               "pressure_dbar":"median",
               "depth_m":"median",
               "temperature_C":"median",
               "odba":"median",
               "speed_ms":"median",
               "heading_deg":"median",
               "pH":"median",
               "battery_soc_%":"mean",
               "capacitive":"mean"
           }))

print("TAG agregado por bin (<=10 filas):\n", tag_bin.head(10), "\n")

# Helpers para unir ENV (en este ejemplo, aproximamos por redondeo y piso semanal)
def _round_keys(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["lat_r"] = out["lat"].round(1)
    out["lon_r"] = out["lon"].round(1)
    out["t_r"]   = pd.to_datetime(out["time_bin"]).dt.floor("7D")
    return out

def merge_env(base: pd.DataFrame, env_df: pd.DataFrame, val_col: str) -> pd.DataFrame:
    b = _round_keys(base)
    e = _round_keys(env_df.rename(columns={"time_bin":"time_bin"}))
    cols = ["lat_r","lon_r","t_r"]
    merged = b.merge(e[cols+[val_col]], on=cols, how="left")
    return merged.drop(columns=[c for c in ["lat_r","lon_r","t_r"] if c in merged.columns])

# Fusiona todas las ENV al TAG-bin
env_merged = tag_bin.copy()
for df_env, col in [
    (env_chl,"CHL_raw"), (env_sst,"SST_raw"), (env_dsst,"dSST_raw"),
    (env_eke,"EKE_raw"), (env_depth,"Depth_raw"), (env_light,"Light_raw"),
    (effort_map,"Effort"), (maxent_s,"S_maxent")
]:
    env_merged = merge_env(env_merged, df_env, col)

# z-scores de ENV (para el modelo)
for col in ["SST_raw","CHL_raw","dSST_raw","EKE_raw","Depth_raw","Light_raw"]:
    if col in env_merged.columns:
        env_merged[col] = env_merged[col].astype("float32")
        env_merged[col + "_z"] = zscore(env_merged[col])

print("ENV fusionado con TAG-bin (<=10 filas):\n", env_merged.head(10), "\n")


# %% [markdown]
# ## 4) One Big Table (OBT) final – **ENV + TAG**
# Claves: `(lat, lon, time_bin)`.  
# **Esquema OBT (propuesto):**
# ```
# schema_obt = {
#  "lat":"float32","lon":"float32","time_bin":"datetime64[ns]",
#  # ENV crudo
#  "SST_raw":"float32","CHL_raw":"float32","dSST_raw":"float32",
#  "EKE_raw":"float32","Depth_raw":"float32","Light_raw":"float32",
#  # ENV z-score
#  "SST_raw_z":"float32","CHL_raw_z":"float32","dSST_raw_z":"float32",
#  "EKE_raw_z":"float32","Depth_raw_z":"float32","Light_raw_z":"float32",
#  # Priors / sesgo
#  "Effort":"float32","S_maxent":"float32",
#  # TAG agregadas
#  "pressure_dbar":"float32","depth_m":"float32","temperature_C":"float32",
#  "odba":"float32","speed_ms":"float32","heading_deg":"float32",
#  "pH":"float32","battery_soc_%":"float32","capacitive":"float32"
# }
# ```

# %%
schema_obt = {
    "lat":"float32","lon":"float32","time_bin":"datetime64[ns]",
    # ENV crudo
    "SST_raw":"float32","CHL_raw":"float32","dSST_raw":"float32",
    "EKE_raw":"float32","Depth_raw":"float32","Light_raw":"float32",
    # ENV z
    "SST_raw_z":"float32","CHL_raw_z":"float32","dSST_raw_z":"float32",
    "EKE_raw_z":"float32","Depth_raw_z":"float32","Light_raw_z":"float32",
    # Priors/sesgo
    "Effort":"float32","S_maxent":"float32",
    # TAG agregadas
    "pressure_dbar":"float32","depth_m":"float32","temperature_C":"float32",
    "odba":"float32","speed_ms":"float32","heading_deg":"float32",
    "pH":"float32","battery_soc_%":"float32","capacitive":"float32"
}

# Selecciona columnas en el orden final
obt_cols = list(schema_obt.keys())

obt = env_merged.copy()
# Asegura que existan todas las columnas del schema
for c in obt_cols:
    if c not in obt.columns:
        obt[c] = np.nan
obt = obt[obt_cols].copy()
obt = enforce_dtypes(obt, schema_obt)

obt_example = obt.head(20).copy()
print("OBT final (20 filas):\n", obt_example, "\n")


# %% [markdown]
# ## 5) Guardado de ejemplos en ./data

# %%
outdir = Path("./data")
outdir.mkdir(parents=True, exist_ok=True)

def save_csv(name: str, df: pd.DataFrame):
    path = outdir / name
    df.to_csv(path, index=False)
    print(f"Guardado: {path} ({len(df)} filas)")

save_csv("example_env_chl.csv", env_chl)
save_csv("example_env_sst.csv", env_sst)
save_csv("example_env_dsst.csv", env_dsst)
save_csv("example_env_eke.csv", env_eke)
save_csv("example_env_depth.csv", env_depth)
save_csv("example_env_light.csv", env_light)
save_csv("example_effort.csv", effort_map)
save_csv("example_maxent.csv", maxent_s)

save_csv("example_tag_raw.csv", tag)
save_csv("example_tag_agg_bin.csv", tag_bin)

save_csv("example_obt_env_tag.csv", obt_example)

print("\nListo. CSVs creados en ./data/")


ENV – CHL_raw (10 filas):
          lat         lon   time_bin   CHL_raw
0  21.882200    4.669400 2017-11-01  1.180831
1 -53.541500  -91.812698 2017-11-08  0.854543
2 -33.556801  116.726997 2017-11-15  2.504011
3 -37.875401 -103.045303 2017-11-22  2.926142
4 -38.891300   86.928101 2017-11-29  0.946160 

ENV – SST_raw (10 filas):
          lat         lon   time_bin    SST_raw
0  50.054699 -154.264694 2017-11-01  17.787664
1  43.762798   29.965799 2017-11-08  19.605875
2 -33.822899  -94.353699 2017-11-15  16.044535
3  43.935299   95.386902 2017-11-22  21.334948
4  27.690201 -117.492599 2017-11-29  24.062223 

TAG – tabla única (10 filas):
    time_h                date   latitude   longitude  pressure_dbar  \
0     0.0 2017-11-02 00:00:00  32.095402    2.113600     238.344315   
1     1.0 2017-11-02 12:00:00  57.743999   92.066002      52.460751   
2     2.0 2017-11-03 00:00:00 -56.603500   -3.951500      58.186424   
3     3.0 2017-11-03 12:00:00 -47.268299  117.261902     114.139694  

C:\Users\Crist\AppData\Local\Temp\ipykernel_26400\1885882292.py:156: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range("2017-11-02", periods=n_tag, freq="12H")
